In [150]:
import pandas as pd
import numpy as np
import os
import re

DATA_DIR = "../capstone/data/cs-train/"

In [4]:
os.listdir(DATA_DIR)

['invoices-2017-11.json',
 'invoices-2017-12.json',
 'invoices-2018-01.json',
 'invoices-2018-02.json',
 'invoices-2018-03.json',
 'invoices-2018-04.json',
 'invoices-2018-05.json',
 'invoices-2018-06.json',
 'invoices-2018-07.json',
 'invoices-2018-08.json',
 'invoices-2018-09.json',
 'invoices-2018-10.json',
 'invoices-2018-11.json',
 'invoices-2018-12.json',
 'invoices-2019-01.json',
 'invoices-2019-02.json',
 'invoices-2019-03.json',
 'invoices-2019-04.json',
 'invoices-2019-05.json',
 'invoices-2019-06.json',
 'invoices-2019-07.json']

In [6]:
dataframes = [pd.read_json(DATA_DIR + file) for file in os.listdir(DATA_DIR)]

In [7]:
dataframes[0].head()

,country,customer_id,invoice,price,stream_id,times_viewed,year,month,day
0,United Kingdom,13085.0,489434,6.95,85048,12,2017,11,28
1,United Kingdom,NaN,489597,8.65,22130,1,2017,11,28
2,United Kingdom,NaN,489597,1.70,22132,6,2017,11,28
3,United Kingdom,NaN,489597,1.70,22133,4,2017,11,28
4,United Kingdom,NaN,489597,0.87,22134,1,2017,11,28


In [8]:
dataframes[-1].head()

,country,customer_id,invoice,price,stream_id,times_viewed,year,month,day
0,United Kingdom,17841.0,558877,0.83,23167,1,2019,7,1
1,United Kingdom,17841.0,558877,0.42,22893,5,2019,7,1
2,United Kingdom,17841.0,558877,0.42,22420,3,2019,7,1
3,United Kingdom,17841.0,558877,0.42,22419,1,2019,7,1
4,United Kingdom,17841.0,558877,1.45,23034,2,2019,7,1


In [27]:
def describe_dataframes(dataframes):
    df_count = 0
    for df in dataframes:
        print("""
        DataFrame {}
            - N rows: {}
            - N cols: {}
            - Column names: {}
            - Missing values: {}
        """.format(df_count, df.shape[0], df.shape[1], ",".join(df.columns.values), df.isnull().sum().sum()))
        df_count += 1

In [28]:
describe_dataframes(dataframes)


        DataFrame 0
            - N rows: 9502
            - N cols: 9
            - Column names: country,customer_id,invoice,price,stream_id,times_viewed,year,month,day
            - Missing values: 2481
        

        DataFrame 1
            - N rows: 35726
            - N cols: 9
            - Column names: country,customer_id,invoice,total_price,stream_id,times_viewed,year,month,day
            - Missing values: 10987
        

        DataFrame 2
            - N rows: 35682
            - N cols: 9
            - Column names: country,customer_id,invoice,total_price,StreamID,TimesViewed,year,month,day
            - Missing values: 10219
        

        DataFrame 3
            - N rows: 29873
            - N cols: 9
            - Column names: country,customer_id,invoice,price,StreamID,TimesViewed,year,month,day
            - Missing values: 5342
        

        DataFrame 4
            - N rows: 38618
            - N cols: 9
            - Column names: country,customer_id,in

In [173]:
def to_snake_case(string):
    pattern_camel_case = "(\w+)+([A-Z]+\w+)"
    groups_camel = re.findall(pattern_camel_case, string)
    if len(groups_camel) > 0:
        return groups_camel[0][0].lower() + "_" + groups_camel[0][1].lower()
    else:
        return string

In [174]:
[to_snake_case(column) for column in dataframes[0].columns.values]

['country',
 'customer_id',
 'invoice',
 'price',
 'stream_id',
 'times_viewed',
 'year',
 'month',
 'day']

In [192]:
def process_dataframes(dataframes):
    processed = []
    for df in dataframes:
        df_new = df.copy()
        df_new.columns = [to_snake_case(column) for column in df.columns.values]
        if "total_price" in df_new.columns.values:
            df_new.rename(columns={"total_price": "price"}, inplace=True)
        processed.append(df_new)
    return pd.concat(processed)

In [193]:
df_p = process_dataframes(dataframes)

In [194]:
df_p

,country,customer_id,invoice,price,stream_id,times_viewed,year,month,day
0,United Kingdom,13085.0,489434,6.95,85048,12,2017,11,28
1,United Kingdom,NaN,489597,8.65,22130,1,2017,11,28
2,United Kingdom,NaN,489597,1.70,22132,6,2017,11,28
3,United Kingdom,NaN,489597,1.70,22133,4,2017,11,28
4,United Kingdom,NaN,489597,0.87,22134,1,2017,11,28
...,...,...,...,...,...,...,...,...,...
42013,United Kingdom,15628.0,562163,0.85,21544,12,2019,7,31
42014,United Kingdom,15628.0,562163,1.95,47591D,8,2019,7,31
42015,United Kingdom,15628.0,562163,0.83,23167,12,2019,7,31
42016,United Kingdom,15628.0,562163,4.95,22138,3,2019,7,31
